In [308]:
import geopandas as gpd
import pickle
from sklearn.neighbors import BallTree
import numpy as np
from shapely.geometry import LineString
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import compress
import pandas as pd

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
def load_obj(name ):
    with open(root_path + 'WorkingData/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
def save_obj(name , obj, extra=""):
    with open(root_path + 'WorkingData/' + extra + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [46]:
def get_nearest_point(src_points, #Source gdf, the gdf with the points you want to attach distances to
                      candidates, #gdf containing the list of points you want to get the distances to
                      k_neighbors=1 #Return just the closest point
                     ):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
     # removed metric since the default is euclidian (what my coordinates use)
    tree = BallTree(candidates, leaf_size=15)

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)

In [30]:
#Import map of Euston for tests
whichmap = "Milford Haven"

Mapset = load_obj("Clipped5k_"+whichmap)

#Mapset = load_obj("All_Maps")

In [31]:
Mapset.keys()

dict_keys(['All_GB', 'National_Parks', 'LocalAuthorities', 'LSOA', 'GreenSpace', 'Rivers', 'Railway_Lines', 'Woodland_Region', 'Urban_Region', 'Foreshor_Region', 'Ferry_Line', 'Coastline', 'Gridlines', 'Lakes', 'Road_Link', 'MotorwayJunctions', 'Motorway', 'A_Roads', 'B_Roads', 'Other_roads', 'NSPL_gdf', 'LandReg_gdf', 'RailwayStations', 'FerryTerminals', 'BusStops', 'Road_Accidents', 'Schools', 'FireStations'])

In [264]:
left_gdf = Mapset['NSPL_gdf']
right_gdf = Mapset["A_Roads"]

name = "roadNumber"

In [223]:
# Ensure that index in right gdf is formed of sequential numbers
left = left_gdf.copy().reset_index(drop=False)
right = right_gdf.copy().reset_index(drop=True)

#Some of the data frames will have empty geometries so this will drop them
left = left[~(left.is_empty)]
right = right[~(right.is_empty)]


In [229]:
point1.distance(point2)

3527.104478180367

In [315]:
def distance(src_points, candidates, source_name, out_name):
    
    dist_name = "Distance_to_nearest_" + out_name
    name_name = "Name_of_nearest_" + out_name
    
    candidates = candidates.reset_index(drop=True)
    
    dist_df = src_points.geometry.apply(lambda g: candidates.distance(g))
    min_distance = list(dist_df.min(axis=1))
    min_name = list(candidates[source_name].iloc[dist_df.idxmin(axis=1)])
    
    df = pd.DataFrame(list(zip(min_distance, min_name)),
                      columns =[dist_name, name_name],
                     index =src_points.index)
    
    return df

In [316]:
distance(Mapset['NSPL_gdf'], Mapset["A_Roads"], "roadNumber", "A_Road")

,Distance_to_nearest_A_Road,Name_of_nearest_A_Road
1973765,2282.397205,A4076
1974394,91.443802,A4076
1974395,146.395059,A4076
1974396,35.542106,A4076
1974397,148.013143,A4076
...,...,...
1975148,152.736333,A4076
1975149,152.736333,A4076
1975150,152.736333,A4076
1975151,152.736333,A4076


In [281]:
Mapset['NSPL_gdf']

,Postcode_NS,oseast1m,osnrth1m,OAC11,CTY20CD,CED17CD,LAD20CD,WD19CD,HLTHAUCD,NHSER19CD,...,CCG20CDH,CCG20NMW,BUA13NM,BUASD13NM,RU11NM,PFA15NM,CALNCV18NM,STP20NM,STP20CDH,geometry
1973765,SA715AF,189300.0,203800.0,NaN,W99999999,NaN,W06000009,W05000414,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,(pseudo) Wales (not covered),(pseudo) Wales (not covered),(England/Wales) Rural village in a sparse setting,Dyfed-Powys,NaN,NaN,NaN,POINT (189300.000 203800.000)
1974394,SA731AA,191271.0,206725.0,NaN,W99999999,NaN,W06000009,W05000427,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (191271.000 206725.000)
1974395,SA731AB,191327.0,206722.0,NaN,W99999999,NaN,W06000009,W05000427,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (191327.000 206722.000)
1974396,SA731AD,191343.0,207051.0,NaN,W99999999,NaN,W06000009,W05000430,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (191343.000 207051.000)
1974397,SA731AE,191299.0,207158.0,NaN,W99999999,NaN,W06000009,W05000430,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (191299.000 207158.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975148,SA739AB,190419.0,206023.0,NaN,W99999999,NaN,W06000009,W05000426,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (190419.000 206023.000)
1975149,SA739AD,190419.0,206023.0,NaN,W99999999,NaN,W06000009,W05000426,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (190419.000 206023.000)
1975150,SA739AE,190419.0,206023.0,NaN,W99999999,NaN,W06000009,W05000426,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (190419.000 206023.000)
1975151,SA739AF,190419.0,206023.0,NaN,W99999999,NaN,W06000009,W05000426,W11000025,NaN,...,7A2,Bwrdd Iechyd Prifysgol Hywel Dda,Milford Haven BUA,(pseudo) Wales (not covered),(England/Wales) Urban city and town,Dyfed-Powys,NaN,NaN,NaN,POINT (190419.000 206023.000)


In [282]:
Mapset["A_Roads"]

,fictitious,identifier,class,roadNumber,name1,name1_lang,name2,name2_lang,formOfWay,length,primary,trunkRoad,loop,startNode,endNode,structure,nameTOID,numberTOID,function,geometry
1724152,false,02007517-C29C-4E4B-9D00-5163640DB201,A Road,A4076,Steynton Road,None,None,None,Single Carriageway,140,true,true,false,497C1D3A-28BC-4AC6-88C8-595478AB580E,4D1D7A69-374C-48EE-B059-FC290D0BDB4B,None,osgb4000000017184181,osgb5000005125744555,A Road,"LINESTRING Z (191722.750 207692.850 0.000, 191..."
1724155,false,6E6D9F91-CB5D-4B4A-B3D9-0681E5DB50AF,A Road,A4076,None,None,None,None,Single Carriageway,263,true,true,false,E224594B-0053-46A8-A7AE-1303E773B63E,DFBEF4C9-1369-442A-BDD7-535BF083C863,None,None,osgb5000005125744555,A Road,"LINESTRING Z (191712.000 208205.000 0.000, 191..."
1724157,false,075F6960-68D1-4E71-9308-1C66508ABBC2,A Road,A4076,None,None,None,None,Single Carriageway,76,true,true,false,DFBEF4C9-1369-442A-BDD7-535BF083C863,41A86249-4C12-49E1-AFC0-98352E0B6403,None,None,osgb5000005125744555,A Road,"LINESTRING Z (191704.650 207946.290 0.000, 191..."
1724158,false,8F2F71ED-7045-419A-8FC0-D8AE7D6A4EDA,A Road,A4076,None,None,None,None,Single Carriageway,587,true,true,false,68F24800-C930-460C-AE1A-07358D4B2BBC,E224594B-0053-46A8-A7AE-1303E773B63E,None,None,osgb5000005125744555,A Road,"LINESTRING Z (191720.215 208247.000 0.000, 191..."
1724160,false,8A9EC9CC-587F-4ABD-8198-86B274BD1F2C,A Road,A4076,None,None,None,None,Single Carriageway,115,true,true,false,41A86249-4C12-49E1-AFC0-98352E0B6403,B71E3FB6-D808-4F8B-A733-392E7EB70EE2,None,None,osgb5000005125744555,A Road,"LINESTRING Z (191720.640 207872.410 0.000, 191..."
1724162,false,21FF6ED8-B8AC-4B70-B0DC-2D0310713566,A Road,A4076,Steynton Road,None,None,None,Collapsed Dual Carriageway,38,true,true,false,497C1D3A-28BC-4AC6-88C8-595478AB580E,1070FBAC-8C15-40E8-887E-98265C68087F,None,osgb4000000017184181,osgb5000005125744555,A Road,"LINESTRING Z (191722.750 207692.850 0.000, 191..."
1724165,false,B3DA33F7-4193-4279-8CE1-2B54EB937E49,A Road,A4076,None,None,None,None,Collapsed Dual Carriageway,27,true,true,false,1070FBAC-8C15-40E8-887E-98265C68087F,B71E3FB6-D808-4F8B-A733-392E7EB70EE2,None,None,osgb5000005125744555,A Road,"LINESTRING Z (191727.520 207730.450 0.000, 191..."
1724213,false,4B031BFB-D9C4-4C33-810E-4DE0020EA10A,A Road,A4076,Steynton Road,None,None,None,Single Carriageway,166,true,true,false,835D9B80-8EF4-4494-A04D-550D78C05270,3DD927E2-649A-4B5D-ACC4-AAB987F09011,None,osgb4000000017184181,osgb5000005125744555,A Road,"LINESTRING Z (191390.000 207041.000 0.000, 191..."
1724216,false,2809E373-557C-4B88-BEDC-EB8D0FA4257F,A Road,A4076,Steynton Road,None,None,None,Single Carriageway,29,true,true,false,30A4CCEA-E981-491D-A29E-94B1B50EB3BF,835D9B80-8EF4-4494-A04D-550D78C05270,None,osgb4000000017184181,osgb5000005125744555,A Road,"LINESTRING Z (191413.000 207058.000 0.000, 191..."
1724218,false,963F9BA8-A0EC-4174-A79F-9C3904E01F18,A Road,A4076,Steynton Road,None,None,None,Single Carriageway,183,true,true,false,DED8F09F-0548-429D-9D87-411B8F71D135,30A4CCEA-E981-491D-A29E-94B1B50EB3BF,None,osgb4000000017184181,osgb5000005125744555,A Road,"LINESTRING Z (191533.000 207195.000 0.000, 191..."
